In [1]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

partnerDistricts =pd.read_excel("partnerDistrict.xlsx")

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)

data['year'] = data['year'].astype(int)
data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)




# Process rows and build JSON
def process_row(row):
    row_data = row.to_dict()
    if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]

        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        row_data['matchingAES'] = []
            
    if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]

        otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
        # print(otherReferrals)
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        print("no data")
        row_data['otherReferrals'] = []

    return row_data

partner_lookup = dict(zip(partnerDistricts['DISTRICT'], partnerDistricts['PARTNER']))

data['Partner'] = data['District'].map(partner_lookup)


json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))



no data
no data
no data
no data
no data
no data
[
    {
        "AERecordingMonth": 0,
        "AERecordingSite": 0,
        "AERecordingYear": 0.0,
        "District": "MBERENGWA",
        "ReferrelRecordingMonth": 0,
        "ReferrelRecordingSite": 0,
        "ReferrelRecordingYear": 0.0,
        "Site_Name": "MUSUME - 101157 - MISSION HOSPITAL",
        "TotalMCsBYMethod": 25,
        "aeComments": 0,
        "ae_classification": 0,
        "ae_type_code": 0,
        "checkMonthValidation": 0.0,
        "checkYearValidation": 0.0,
        "checkifFacilityHasBeenSaved": 0,
        "circumcising_cadre": 0,
        "client_age": 0.0,
        "date_ae_identified": 0,
        "facilityName": "Negove Clinic",
        "followUpTotal": 0.0,
        "fu15-19": 22,
        "fu20-24": 0.0,
        "fu25-29": 0.0,
        "fu30-34": 0.0,
        "fu35-39": 0,
        "fu40-44": 0.0,
        "fu45-49": 0,
        "fu50": 0,
        "hivNegative15-19": 12,
        "hivNegative20-24": 0.0,
      

In [2]:
data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,aeComments,...,total_surgicalReusable.6,total_surgicalReusable.7,total_surgicalReusable.8,totalhivNegative,totalhivNegativeNC,uncircumcisedClientsForHTS,vmmc_number,year,facilityType,Partner
0,0,0,0.0,MBERENGWA,0,0,0.0,MUSUME - 101157 - MISSION HOSPITAL,25,0,...,OK,0,0,12,0,0.0,0.0,2025,0,ZACH
1,November,Negove Clinic,2025.0,MBERENGWA,November,Negove Clinic,2025.0,MUSUME - 101157 - MISSION HOSPITAL,35,0,...,OK,0,0,28,0,0.0,0.0,2025,0,ZACH
2,January,Mbembeswana,2025.0,MATOBO,January,Mbembeswana,2025.0,MAPHISA - 100890 - DISTRICT HOSPITAL,35,Client is healing,...,OK,OK,0,34,0,1.0,789.0,2025,Outreach,ZICHIRE
3,0,0,0.0,MATOBO,January,Beula,2025.0,MAPHISA - 100890 - DISTRICT HOSPITAL,45,0,...,OK,0,0,35,0,0.0,0.0,2025,Outreach,ZICHIRE
4,0,0,0.0,GURUVE,0,0,0.0,GURUVE - 100553 - DISTRICT HOSPITAL,9,0,...,OK,0,OK,9,0,0.0,0.0,2025,0,ZICHIRE
5,0,0,0.0,GURUVE,0,0,0.0,GURUVE - 100553 - DISTRICT HOSPITAL,25,0,...,OK,0,0,25,0,0.0,0.0,2025,0,ZICHIRE
6,0,0,0.0,GURUVE,January,Birkdale,2025.0,GURUVE - 100553 - DISTRICT HOSPITAL,36,0,...,OK,0,0,34,0,0.0,0.0,2025,0,ZICHIRE
7,0,0,0.0,GURUVE,December,Gota,2024.0,GURUVE - 100553 - DISTRICT HOSPITAL,15,0,...,OK,OK,0,15,1,1.0,0.0,2025,0,ZICHIRE
8,December,Svita Clinic,2024.0,MBERENGWA,December,Svita Clinic,2024.0,MUSUME - 101157 - MISSION HOSPITAL,43,Client is healing,...,OK,0,0,43,0,0.0,585858.0,2024,0,ZACH
9,0,0,0.0,MBERENGWA,December,Neta Clinic,2024.0,MUSUME - 101157 - MISSION HOSPITAL,45,0,...,OK,0,0,44,0,0.0,0.0,2024,0,ZACH
